In [1]:
import pandas as pd

In [2]:
class EgyptianNationalIDValidator:
    def __init__(self, nationalId):
        self.nationalId = nationalId
        self.valid = self.validateNationalId()
        if self.valid:
            self.birthDate = self.extractBirthDate()
            self.governorate = self.extractGovernorate()
            self.gender = self.extractGender()
        else:
            self.birthDate = None
            self.governorate = None
            self.gender = None

    def validateNationalId(self):
        if len(self.nationalId) != 14 or not self.nationalId.isdigit():
            return False
        
        #"3 01 05 17 0104863" # 2 = 1900 and 3 = 2000
        # birth date 
        century = int(self.nationalId[0])
        year = int(self.nationalId[1:3])
        month = int(self.nationalId[3:5])
        day = int(self.nationalId[5:7])
        
        
        if century not in [2, 3]:  
            return False
        if month < 1 or month > 12:
            return False
        if day < 1 or day > 31:
            return False      
        if month in [4, 6, 9, 11] and day > 30:
            return False
        if month == 2:
            birthYear = 1900 + year if century == 2 else 2000 + year
            isLeapYear = (birthYear % 4 == 0 and birthYear % 100 != 0) or (birthYear % 400 == 0)
            if day > 29 if isLeapYear else day > 28:
                return False
        return True

    def extractBirthDate(self):
        century = int(self.nationalId[0])
        year = int(self.nationalId[1:3])
        month = int(self.nationalId[3:5])
        day = int(self.nationalId[5:7])
        
        if century == 2:
            birthYear = 1900 + year
        else:
            2000 + year
                
        return f"{birthYear}/{month:02d}/{day:02d}"
    

    def extractGovernorate(self):
        #"3010517 01 04863"
        governorateCode = self.nationalId[7:9]
        governorateMapping = {"01": "Cairo",
                              "02": "Alexandria",
                              "03": "Port Said",
                              "04": "Suez", }
        return governorateMapping.get(governorateCode, "Unknown Governorate")
    
    def extractGender(self):
        #"3010517010486 3 "
        genderDigit = int(self.nationalId[12])
        if genderDigit % 2 != 0: 
            return "Female"   
        else:
            return "Male"

    def getInfo(self):
        if not self.valid:
            return {"valid": False, "message": "Invalid national ID."}
        
        return {
            "valid": True,
            "birth_date": self.birthDate,
            "governorate": self.governorate,
            "gender": self.gender,
            "message": "National ID is valid."
        }

In [3]:
# Test with a valid national ID
nationalId = "29001011234567"
validator = EgyptianNationalIDValidator(nationalId)
result = validator.getInfo()
print(result)

{'valid': True, 'birth_date': '1990/01/01', 'governorate': 'Unknown Governorate', 'gender': 'Male', 'message': 'National ID is valid.'}


In [4]:
!pip install djangorestframework

In [5]:
!python manage.py startapp id_validator

python: can't open file 'C:\Users\diman\manage.py': [Errno 2] No such file or directory


In [7]:
import rest_framework
print("Django REST Framework is installed!")

import sys
print(sys.executable)

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',  
]

from django.core.management import call_command

import os
import django
from django.conf import settings

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

# Configure Django settings
if not settings.configured:
    settings.configure(
        INSTALLED_APPS=[
            'django.contrib.auth',
            'django.contrib.contenttypes',
            'rest_framework',
        ],
        REST_FRAMEWORK={
            'DEFAULT_THROTTLE_CLASSES': [
                'rest_framework.throttling.UserRateThrottle',
            ],
            'DEFAULT_THROTTLE_RATES': {
                'user': '100/day',
            },
        },
    )

# Initialize Django
django.setup()

# Now you can use DRF
from rest_framework.throttling import UserRateThrottle

class CustomThrottle(UserRateThrottle):
    rate = '100/day'

print("Django and DRF are configured!")

Django REST Framework is installed!
C:\Users\diman\anaconda3\python.exe
Django and DRF are configured!


In [8]:
from rest_framework.views import APIView
from rest_framework.response import Response

class ValidateNationalIDView(APIView):
    throttle_classes = [CustomThrottle]

    def post(self, request):
        nationalId = request.data.get('nationalId')
        validator = EgyptianNationalIDValidator(nationalId)
        return Response(validator.getInfo())

In [9]:
from rest_framework.authentication import BaseAuthentication
from rest_framework.exceptions import AuthenticationFailed

class APIKeyAuthentication(BaseAuthentication):
    def authentication(self, request):
        apiKey = request.META.get('HTTP_X_API_KEY')
        if not apiKey:
            return None
        try:
            user = User.objects.get(apiKey=apiKey)
        except User.DoesNotExist:
            raise AuthenticationFailed('Invalid API key.')
        return (user, None)

In [10]:
!python manage.py startapp id_validator

python: can't open file 'C:\Users\diman\manage.py': [Errno 2] No such file or directory


In [11]:
from django.db import models
from django.contrib.auth.models import User

class APICallLog(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    national_id = models.CharField(max_length=14)
    timestamp = models.DateTimeField(auto_now_add=True)

    class Meta:
        app_label = 'id_validator'  

In [12]:
def validate_national_id(request):
    APICallLog.objects.create(user=request.user, national_id=national_id)